In [1]:
%matplotlib inline


# 10 - Tracking in simulation: bringing all components together
The previous tutorials have introduced various aspects of Stone Soup covering inference and data
association for multiple-target trackers, using simulated data. This tutorial consolidates those
aspects in a notebook which can be modified to individual need. It contains all aspects
introduced in previous tutorials, and nothing new.


## Process
This notebook, as with the previous, proceeds according to the following steps:

1. Create the simulation

  * Initialise the 'playing field'
  * Choose number of targets and initial states
  * Create some transition models
  * Create some sensor models

2. Initialise the tracker components

  * Initialise predictors
  * Initialise updaters
  * Initialise data associations, hypothesisers
  * Initiators and deleters
  * Create the tracker

3. Run the tracker

  * Plot the output




## Create the simulation



Separate out the imports



In [2]:
import numpy as np
import datetime

### Initialise ground truth
Here are some configurable parameters associated with the ground truth, e.g. defining where
tracks are born and at what rate, death probability. This follows similar logic to the code
in previous tutorial section `auto_tutorials/09_Initiators_&_Deleters:Simulating Multiple
Targets`.



In [3]:
from stonesoup.types.array import StateVector, CovarianceMatrix
from stonesoup.types.state import GaussianState
initial_state_mean = StateVector([[0], [0], [0], [0]])
initial_state_covariance = CovarianceMatrix(np.diag([4, 0.5, 4, 0.5]))
timestep_size = datetime.timedelta(seconds=5)
number_of_steps = 20
birth_rate = 0.3
death_probability = 0.05
initial_state = GaussianState(initial_state_mean, initial_state_covariance)

## Transition model

Create the transition model - default set to 2d nearly-constant velocity with small (0.05)
variance.



In [4]:
from stonesoup.models.transition.linear import (
    CombinedLinearGaussianTransitionModel, ConstantVelocity)
transition_model = CombinedLinearGaussianTransitionModel(
    [ConstantVelocity(0.05), ConstantVelocity(0.05)])

Put this all together in a multi-target simulator.



In [5]:
from stonesoup.simulator.simple import MultiTargetGroundTruthSimulator
groundtruth_sim = MultiTargetGroundTruthSimulator(
    transition_model=transition_model,
    initial_state=initial_state,
    timestep=timestep_size,
    number_steps=number_of_steps,
    birth_rate=birth_rate,
    death_probability=death_probability
)

### Initialise the measurement models
The simulated ground truth will then be passed to a simple detection simulator. This again has a
number of configurable parameters, e.g. where clutter is generated and at what rate, and
detection probability. This implements similar logic to the code in the previous tutorial section
`auto_tutorials/09_Initiators_&_Deleters:Generate Detections and Clutter`.



In [6]:
from stonesoup.simulator.simple import SimpleDetectionSimulator
from stonesoup.models.measurement.linear import LinearGaussian

# initialise the measurement model
measurement_model_covariance = np.diag([0.25, 0.25])
measurement_model = LinearGaussian(4, [0, 2], measurement_model_covariance)

# probability of detection
probability_detection = 0.9

# clutter will be generated uniformly in this are around the target
clutter_area = np.array([[-1, 1], [-1, 1]])*30
clutter_rate = 1

The detection simulator



In [7]:
detection_sim = SimpleDetectionSimulator(
    groundtruth=groundtruth_sim,
    measurement_model=measurement_model,
    detection_probability=probability_detection,
    meas_range=clutter_area,
    clutter_rate=clutter_rate
)

## 2d lidar person detection test

In [133]:
# load logged file
# format: list of detections, each detctions is a nx3 numpy arra (detx,dety,detprob)

#filename = "/home/pblnav/logs/detections_11-19-2022-05-55-44.npz"
filename = "/home/pblnav/logs/detections_11-19-2022-05-56-29.npz"

data = np.load(filename)

person_detections = [data[k] for k in data]

for p in person_detections:
    #print(p)
    for d in p:
        print(d)
    break


[ 5.9869463  -2.61394129  0.99998772]
[1.39674245 0.6150559  0.99980503]
[ 5.79414298 -1.6730116   0.99971348]
[ 6.63743661 -1.32559474  0.99288863]
[ 0.35854218 -1.53810788  0.98771149]
[-0.21337207  1.76440395  0.97714806]


### Idea: wrap 2d lidar person detector detection in a stonesoup.detector.base.Detector

In [134]:
import threading
from abc import abstractmethod
from copy import copy
from datetime import timedelta

from stonesoup.detector.base import Detector
from stonesoup.base import Property
from stonesoup.buffered_generator import BufferedGenerator
from stonesoup.types.detection import Detection



class LidarPersonDetector(Detector):
    """ Stone Soup Detector wraper for 2d point detections (2d lidar person detector)
    Copied from Video Async Detector Abstract Class
   
    """
    
    person_detections: list = Property(
        doc="person_detections: generator or array of detections for each loop.  each Detection = Nx3 numpy (x,y,confidence)",
        default=False)

    def __init__(self,*args, **kwargs):
        super().__init__(None,*args, **kwargs)
        """
        Args:
            person_detections:
                generator or array of detections for each loop.  each Detection = Nx3 numpy (x,y,confidence)
        """ 
        self.person_detections = person_detections


    @BufferedGenerator.generator_method
    def detections_gen(self):
        """Returns a generator of detections for each frame.
        Yields
        ------
        : :class:`datetime.datetime`
            Datetime of current time step
        : set of :class:`~.Detection`
            Detections generated in the time step. The detection state vector is of the form
            ``(x, y)``, position of detection. Additionally, each detection carries the
            following meta-data fields:
            - ``confidence``: A float in the range ``(0, 1]`` indicating the detector's confidence
        """
        timestamp = datetime.datetime.now()
              
        for person_det in person_detections:
            
            detections = set()
            
            timestamp += timedelta(seconds=0.1)
            
            for det in person_det:
                
                state_vector = StateVector([det[0], det[1]])
                metadata = {"confidence": det[2]}
                
                detection = Detection(state_vector=state_vector, #StateVector
                                  timestamp=timestamp, #datetime.datetime
                                  metadata=metadata) #dictionary of metadata items
                detections.add(detection)
                
            
            yield timestamp, detections

In [135]:
detector = LidarPersonDetector(person_detections=person_detections)
for det in detector:
    print(det)

(datetime.datetime(2022, 11, 19, 18, 34, 55, 918096), {Detection(
    state_vector=StateVector([[ 6.63743661],
                              [-1.32559474]]),
    timestamp=2022-11-19 18:34:55.918096,
    measurement_model=None,
    metadata={'confidence': 0.9928886294364929}), Detection(
    state_vector=StateVector([[-0.21337207],
                              [ 1.76440395]]),
    timestamp=2022-11-19 18:34:55.918096,
    measurement_model=None,
    metadata={'confidence': 0.9771480560302734}), Detection(
    state_vector=StateVector([[ 5.9869463 ],
                              [-2.61394129]]),
    timestamp=2022-11-19 18:34:55.918096,
    measurement_model=None,
    metadata={'confidence': 0.9999877214431763}), Detection(
    state_vector=StateVector([[1.39674245],
                              [0.6150559 ]]),
    timestamp=2022-11-19 18:34:55.918096,
    measurement_model=None,
    metadata={'confidence': 0.9998050332069397}), Detection(
    state_vector=StateVector([[ 5.79414298],

### Transition model

We begin our definition of the state-space models by defining the hidden state
$\mathrm{x}_k$, i.e. the state that we wish to estimate:

\begin{align}\mathrm{x}_k = [x_k, \dot{x}_k, y_k, \dot{y}_k]\end{align}

where $x_k, y_k$ denote center or detection, with $\dot{x}_k, \dot{y}_k$ denoting their respective rate of change

In [136]:
from stonesoup.models.transition.linear import (
    CombinedLinearGaussianTransitionModel, ConstantVelocity)

transition_model = CombinedLinearGaussianTransitionModel(
    [ConstantVelocity(0.05), ConstantVelocity(0.05)])

In [137]:
#from stonesoup.models.transition.linear import (CombinedLinearGaussianTransitionModel,
#                                                ConstantVelocity, RandomWalk)
#t_models = [ConstantVelocity(20**2), ConstantVelocity(20**2), RandomWalk(20**2), RandomWalk(20**2)]
#transition_model = CombinedLinearGaussianTransitionModel(t_models)

### Measurement model

In [138]:
from stonesoup.models.measurement.linear import LinearGaussian

measurement_model_covariance = np.diag([0.25, 0.25])
measurement_model = LinearGaussian(ndim_state=4, mapping=[0, 2],noise_covar=measurement_model_covariance)

## Create the tracker components
In this example a Kalman filter is used with global nearest neighbour (GNN) associator. Other
options are, of course, available.




### Predictor
Initialise the predictor using the same transition model as generated the ground truth. Note you
don't have to use the same model.



In [139]:
from stonesoup.predictor.kalman import KalmanPredictor
predictor = KalmanPredictor(transition_model)

### Updater
Initialise the updater using the same measurement model as generated the simulated detections.
Note, again, you don't have to use the same model (noise covariance).



In [140]:
from stonesoup.updater.kalman import KalmanUpdater
updater = KalmanUpdater(measurement_model)

### Data associator
Initialise a hypothesiser which will rank predicted measurement - measurement pairs according to
some measure.
Initialise a Mahalanobis distance measure to facilitate this ranking.



In [141]:
from stonesoup.hypothesiser.distance import DistanceHypothesiser
from stonesoup.measures import Mahalanobis
hypothesiser = DistanceHypothesiser(predictor, updater, measure=Mahalanobis(), missed_distance=3)

Initialise the GNN with the hypothesiser.



In [142]:
from stonesoup.dataassociator.neighbour import GNNWith2DAssignment
data_associator = GNNWith2DAssignment(hypothesiser)

### Initiator and Deleter
Create deleter - get rid of anything with a covariance trace greater than 2



In [143]:
from stonesoup.deleter.error import CovarianceBasedDeleter
covariance_limit_for_delete = 2
deleter = CovarianceBasedDeleter(covar_trace_thresh=covariance_limit_for_delete)

Set a standard prior state and the minimum number of detections required to qualify for
initiation



In [144]:
s_prior_state = GaussianState([[0], [0], [0], [0]], np.diag([0, 0.5, 0, 0.5]))
min_detections = 3

Initialise the initiator - use the 'full tracker' components specified above in the initiator.
But note that other ones could be used if needed.



In [145]:
from stonesoup.initiator.simple import MultiMeasurementInitiator
initiator = MultiMeasurementInitiator(
    prior_state=s_prior_state,
    measurement_model=measurement_model,
    deleter=deleter,
    data_associator=data_associator,
    updater=updater,
    min_points=min_detections
)

## Run the Tracker
With the components created, the multi-target tracker component is created, constructed from
the components specified above. This is logically the same as tracking code in the previous
tutorial section `auto_tutorials/09_Initiators_&_Deleters:Running the Tracker`



In [146]:
from stonesoup.tracker.simple import MultiTargetTracker

tracker = MultiTargetTracker(
    initiator=initiator,
    deleter=deleter,
    detector=detector,
    data_associator=data_associator,
    updater=updater,
)

In the case of using (J)PDA like in `auto_tutorials/07_PDATutorial:Run the PDA Filter`
and `auto_tutorials/08_JPDATutorial:Running the JPDA filter`, then the
:class:`~.MultiTargetMixtureTracker` would be used instead on the
:class:`~.MultiTargetTracker` used above.

### Plot the outputs
We plot the ground truth, detections and the tracker output using the Stone Soup :class:`Plotter`.



In [147]:
groundtruth = set()
detections = set()
tracks = set()

for time, ctracks in tracker:
    #groundtruth.update(groundtruth_sim.groundtruth_paths)
    detections.update(detector.detections)
    tracks.update(ctracks)

In [148]:
from stonesoup.plotter import Plotterly

plotter = Plotterly()
#plotter.plot_ground_truths(groundtruth, mapping=[0, 2])
plotter.plot_measurements(detections, mapping=[0, 2])
plotter.plot_tracks(tracks, mapping=[0, 2])
plotter.fig